Importing the libraries

In [11]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

'2.9.1'

Part 1 - Data Preprocessing

--Preprocessing the Training set

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory("imageData2\\train",
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 33232 images belonging to 27 classes.


Preprocessing the Test set

In [13]:

test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('imageData2\\test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 8308 images belonging to 27 classes.


Part 2 - Building the CNN

--Initialising the CNN

In [14]:
cnn = tf.keras.models.Sequential()

Step 1 - Convolution

In [25]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

Step 2 - Pooling

In [26]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a second convolutional layer

In [27]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Step 3 - Flattening

In [28]:
cnn.add(tf.keras.layers.Flatten())

Step 4 - Full Connection

In [29]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

Step 5 - Output Layer

In [30]:
cnn.add(tf.keras.layers.Dense(units=27, activation='softmax'))

Part 3 - Training the CNN

--Compiling the CNN

In [31]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

Training the CNN on the Training set and evaluating it on the Test set

In [32]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 15)

Epoch 1/15
1039/1039 [==============================] - 234s 225ms/step - loss: 1.1639 - accuracy: 0.6650 - val_loss: 0.6710 - val_accuracy: 0.8101
Epoch 2/15
1039/1039 [==============================] - 207s 199ms/step - loss: 0.5879 - accuracy: 0.8193 - val_loss: 0.4890 - val_accuracy: 0.8581
Epoch 3/15
1039/1039 [==============================] - 212s 204ms/step - loss: 0.4422 - accuracy: 0.8636 - val_loss: 0.3773 - val_accuracy: 0.8894
Epoch 4/15
1039/1039 [==============================] - 204s 197ms/step - loss: 0.3560 - accuracy: 0.8891 - val_loss: 0.3507 - val_accuracy: 0.8961
Epoch 5/15
1039/1039 [==============================] - 205s 198ms/step - loss: 0.3147 - accuracy: 0.9006 - val_loss: 0.3179 - val_accuracy: 0.9049
Epoch 6/15
1039/1039 [==============================] - 192s 185ms/step - loss: 0.2750 - accuracy: 0.9121 - val_loss: 0.3204 - val_accuracy: 0.9117
Epoch 7/15
1039/1039 [==============================] - 162s 156ms/step - loss: 0.2430 - accuracy: 0.9223 - val_

Part 4 - Making a single prediction

In [15]:
training_set.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25,
 'space': 26}

create model file

In [57]:
cnn.save('cnn_model_4.h5')

summary of the model

In [16]:
model = tf.keras.models.load_model('cnn_model_4.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

one image test

In [17]:
import cv2
import numpy as np
from keras.preprocessing import image
test_image = tf.keras.utils.load_img('imageData2\\test\E\e_1472.jpg', target_size = (64, 64))
model = tf.keras.models.load_model('cnn_model_4.h5')
#test_image= tf.image.resize_with_pad(test_image, 64,64)

In [18]:
test_image = tf.keras.utils.img_to_array(test_image)
print(test_image.shape)
test_image = np.expand_dims(test_image, axis = 0)
print(test_image.shape)
result = model.predict(test_image)
print(result)

(64, 64, 3)
(1, 64, 64, 3)
1/1 [==============================] - 0s 117ms/step
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]]


In [19]:
letter_position = 0
for i in result[0]:   
    if i != 0:
        print(i)
        position = list(training_set.class_indices.values()).index(letter_position)
        print(position)
        print(list(training_set.class_indices.keys())[position])
    letter_position += 1

1.0
4
E
